In [ ]:
import numpy as np
import cv2
import os

In [ ]:
dataset_name = "./Judol Detection"

In [ ]:

test_path = dataset_name + '/test'
images_path = os.path.join(test_path, 'images')
labels_path = os.path.join(test_path, 'labels')
labels_name = ['BK8', 'Gate-of-olympus', 'Princess', 'Starlight-Princess', 'Zeus']


face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


def prepare_data(images_path, labels_path):
    face_samples = []
    ids = []

    for label_file in os.listdir(labels_path):
        label_file_path = os.path.join(labels_path, label_file)
        with open(label_file_path, 'r') as f:
            label_data = f.readlines()


        img_file = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(images_path, img_file)

        img = cv2.imread(img_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        for line in label_data:
            data = line.strip().split()
            label = int(data[0])
            x_center, y_center, width, height = map(float, data[1:])


            img_height, img_width = gray.shape
            x_center *= img_width
            y_center *= img_height
            width *= img_width
            height *= img_height


            x_start = int(x_center - width / 2)
            y_start = int(y_center - height / 2)
            x_end = int(x_center + width / 2)
            y_end = int(y_center + height / 2)


            face = gray[y_start:y_end, x_start:x_end]


            detected_faces = face_detector.detectMultiScale(face)
            for (x, y, w, h) in detected_faces:
                face = face[y:y+h, x:x+w]
                break


            face = cv2.resize(face, (150, 150))


            face_samples.append(face)
            ids.append(label)

    return face_samples, ids


def train_recognizer(face_samples, ids):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(face_samples, np.array(ids))
    return recognizer


In [ ]:
# Main execution
if __name__ == "__main__":
    if os.path.exists("./lbph_recognizer.yml"):
       # Load trained LBPH recognizer
      recognizer = cv2.face.LBPHFaceRecognizer_create()
      model_path = './lbph_recognizer.yml'
      recognizer.read(model_path)
    else:
      print("Preparing data...")
      faces, labels = prepare_data(images_path, labels_path)
      print(f"Data prepared: {len(faces)} faces")
      print("Training recognizer...")
      recognizer = train_recognizer(faces, labels)
      print("Recognizer trained successfully.")
      # Save model
      model_path = './lbph_recognizer.yml'
      recognizer.write(model_path)
      print(f"Model saved to {model_path}")

In [ ]:
import os
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


def test_recognizer(images_path, labels_path):
    correct_predictions = 0
    total_images = 0
    total_predicted_boxes = 0
    total_true_boxes = 0

    for label_file in os.listdir(labels_path):
        label_file_path = os.path.join(labels_path, label_file)
        with open(label_file_path, 'r') as f:
            label_data = f.readlines()


        img_file = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(images_path, img_file)


        img = cv2.imread(img_path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


        detected_faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        predicted_labels = []
        true_labels = []
        predicted_bboxes = []
        true_bboxes = []


        for (x, y, w, h) in detected_faces:
            predicted_bboxes.append((x, y, w, h))
            predicted_labels.append(-1)


        for line in label_data:
            data = line.strip().split()
            true_label = int(data[0])
            x_center, y_center, width, height = map(float, data[1:])


            img_height, img_width = gray.shape
            x_center *= img_width
            y_center *= img_height
            width *= img_width
            height *= img_height

            x_start = int(x_center - width / 2)
            y_start = int(y_center - height / 2)
            x_end = int(x_center + width / 2)
            y_end = int(y_center + height / 2)

            true_bboxes.append((x_start, y_start, x_end, y_end))
            true_labels.append(true_label)


        for pred_box, true_box, true_label in zip(predicted_bboxes, true_bboxes, true_labels):

            x1, y1, x2, y2 = pred_box
            tx1, ty1, tx2, ty2 = true_box


            x1_inter = max(x1, tx1)
            y1_inter = max(y1, ty1)
            x2_inter = min(x2, tx2)
            y2_inter = min(y2, ty2)
            inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
            pred_area = (x2 - x1) * (y2 - y1)
            true_area = (tx2 - tx1) * (ty2 - ty1)

            iou = inter_area / float(pred_area + true_area - inter_area)


            if iou > 0.5 and true_label == predicted_labels[0]:
                correct_predictions += 1

            total_true_boxes += 1
            total_predicted_boxes += 1


        for pred_box, true_box, pred_label, true_label in zip(predicted_bboxes, true_bboxes, predicted_labels, true_labels):
            x1, y1, x2, y2 = pred_box
            tx1, ty1, tx2, ty2 = true_box


            bbox_area = (x2 - x1) * (y2 - y1)
            img_area = img.shape[0] * img.shape[1]
            confidence = bbox_area / img_area


            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

            cv2.rectangle(img, (tx1, ty1), (tx2, ty2), (0, 255, 0), 2)


            predicted_label_name = labels_name[pred_label] if pred_label != -1 else "Zeus"
            pred_label_text = f"Pred: {predicted_label_name}, Conf: {confidence:.2f}"
            cv2.putText(img, pred_label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


            true_label_name = labels_name[true_label]
            true_label_text = f"True: {true_label_name}"
            cv2.putText(img, true_label_text, (tx1, ty1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


        cv2.imshow("Result", img)
        cv2.waitKey(0)


        result_path = os.path.join(test_path, 'results', img_file)
        if not os.path.exists(os.path.dirname(result_path)):
            os.makedirs(os.path.dirname(result_path))
        cv2.imwrite(result_path, img)

    cv2.destroyAllWindows()


    accuracy = (correct_predictions / total_true_boxes) * 100 if total_true_boxes > 0 else 0
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Total predicted boxes: {total_predicted_boxes}")
    print(f"Total true boxes: {total_true_boxes}")


if __name__ == "__main__":
    print("Testing the model on the test dataset...")
    test_recognizer(images_path, labels_path)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern


def compute_lbp(image, radius=1, neighbors=8):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    lbp = local_binary_pattern(gray, neighbors, radius, method='uniform')

    return lbp


def visualize_and_save_lbph_texture(image_path, output_dir="output"):

    image = cv2.imread(image_path)


    lbp = compute_lbp(image)


    original_filename = f"{output_dir}/original_{image_path.split('/')[-1]}"
    lbp_filename = f"{output_dir}/lbp_{image_path.split('/')[-1]}"


    cv2.imwrite(original_filename, image)
    cv2.imwrite(lbp_filename, lbp)


    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(lbp, cmap='gray')
    plt.title("LBP Texture")
    plt.axis("off")

    plt.show()

    print(f"Processed images saved to: {original_filename}, {lbp_filename}")
    return original_filename, lbp_filename


image_path_real = "real.webp"
image_path = "zeus.jpg"


output_dir = "output"


original_real, lbp_real = visualize_and_save_lbph_texture(image_path_real, output_dir)
original_zeus, lbp_zeus = visualize_and_save_lbph_texture(image_path, output_dir)
